# Pseudo Random
## Produced by: Ruben Girela Castellón

## Postulates Golomb
The postulates Golomb are neccesary but not sufficient conditions for pseudorandom sequences to appear random.

Its operation is simple:

Give a sequence $S=\{S^0,S^1,...,S^n$ of period **n**, the postulates they pose are:

1. In the cycle $S^n$ of **S**, the difference of 1s and 0s that there is at most has to be 1.
2. In the cycle $S^n$, the various runs are of length $\frac{1}{2^n}$ exactly, i.e. half runs of length 1, quarter runs of length 2, eighth runs of length 3, ...
3. We cyclically shift the sequence $S^n$ by one bit and calculate the Hamming distance between the new sequence and the original, if the distance is the same as the previous one, the process is repeated, in otherwise it ends and it is verified that the last value obtained is 0. If it is 0, it fulfills the third postulate, otherwise it does not. 

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  6 18:14:10 2022

@author: Ruben Girela Castellón
"""
import numpy as np

In [2]:
#función que determina si una secuencia de bits cumple los postulados de Golomb
def postGolomb(bits):
    
    #1º comprobamos que la cantidad de 1s y 0s difieren como mucho en 1
    #para ello contamos el numero de 0s y 1s
    ceros = bits.count('0')
    unos = bits.count('1')
    
    #compruebo la diferencia absoluta entre 0s y 1s sean 0 o 1 como mucho.
    if(np.abs(ceros-unos) <= 1):
        
        '''
        iniciamos la racha anterior y la siguiente como el maximo entre los 
        contadores de 0s y 1s y rachas_next como la mitad de la anterior.
        '''
        rachas_last = max(ceros, unos)
        rachas_next = rachas_last//2
        
        #contador en el que el numero de n-rachas sea 1
        contador_unos = 0
        
        #iniciamos el numero de rachas a 0
        rachas = 0     
        
        #comprobamos si el primer bit y el ultimo no son distintos
        while(bits[0] == bits[-1]):
            #si son iguales rotamos hasta que el primero sea distinto al ultimo
            bits = bits[-1]+bits[:-1]
        

        '''
        2º comprobamos de las diversas rachas son de longitud 1/2^n, siendo n el 
        numero de rachas, mientras el contador de n-rachas siguiente sea >= que 
        la mitad entera del anterior o el contador de unos es > 0 y < 3 y el 
        contador de n-rachas siguientes no sea 0.
        '''
        while(
                (rachas_next == rachas_last//2 or 
                 (contador_unos < 3 and contador_unos > 0)
                ) and rachas_next > 0):
            
            #si el numero de n-rachas es 1 incrementamos el contador de unos
            if(rachas_next == 1): contador_unos +=1
            
            #incremento el numero de rachas
            rachas += 1
            #contador de rachas que será modificado
            rachas_m = rachas
            
            #si el numero de rachas es 2 o más
            if(rachas > 1):
                #actualizo rachas_last
                rachas_last = rachas_next
                
            #reseteamos rachas_next
            rachas_next = 0
                
            #inicializamos el posicionador de bits
            i = 0
            
            #mientras no supere el rango de la secuencia de bits
            while(i<len(bits)):
                
                #obtengo el primer bit distinto con el anterior
                bit = bits[i]
                    
                #incrementamos i
                i += 1

                #decrementamos el contador de rachas
                rachas_m -= 1

                #si no supera el rango de bits
                if(i < len(bits)):

                    '''
                    si el numero de rachas es 2 o mas o el bit siguiente es 
                    igual al anterior.
                    '''
                    if(rachas_m>0 or bits[i] == bit):

                        #avanza hasta el siguiente bit distinto
                        while(i < len(bits) and bits[i]==bit):

                            #decrementando el contador de rachas
                            rachas_m -= 1

                            #e incrementando la posición
                            i += 1

                #si el numero de rachas es 0
                if(rachas_m == 0):
                    #incrementamos el contador de n-rachas
                    rachas_next += 1

                #reseteamos el contador de rachas
                rachas_m = rachas  
            
            #si la racha siguiente es impar y es mayor a 1, no cumple el 2º postulado
            if(rachas_next%2 != 0 and rachas_next > 1): return False
            
            #si el numero de n-rachas es 0 y el contador de unos no supera a 2
            if(rachas_next == 0 and contador_unos <= 2 and rachas > 1):
                
                '''
                Cuando se cumple el 2º postulado, comprobamos que se cumple el 
                3º postulado, que consta de desplazar un bit de forma ciclica y 
                calcular la distancia de Hamming de esa nueva secuencia con la 
                original, se compara la distancia Hamming anterior con la siguiente
                , si son iguales se repite el proceso, hasta que sean distintos, 
                si es 0 cumple el 3º postulado, en caso contrario no.
                '''
                #para ello desplazamos un bit a la derecha.
                #Ejemplo: 1001 --> 0011
                #convertimos la secuencia en un array de enteros
                b1 = np.array(list(map(int,bits)))
                #desplazo la secuencia de derecha a izquierda
                b2 = b1[1:]
                b2 = np.append(b2,b1[0])
                
                #calculo la primera diferencia
                after_diff = np.sum(np.abs(b1-b2))
                #se pone el mismo valor que el anterior para que entre en el bucle
                diff = after_diff
                
                '''
                mientras sea la diferencia igual a la anterior itera hasta que 
                sea distinto:
                    - 0 (ha dado la vuelta a toda la secuencia)
                    - x (no cumple el 3º postulado)
                '''
                while(after_diff == diff):
                    
                    #desplazo la secuencia de derecha a izquierda
                    b2 = np.append(b2,b2[0])
                    b2 = b2[1:]
                    
                    #calculo la distancia hamming
                    diff = np.sum(np.abs(b1-b2))
                    
                #si la diferencia es 0 es que cumple los postulados de Golomb
                if(diff == 0): return True
                #en caso contrario no
                return False
    
    #en caso contrario no
    return False

Some examples:

In [3]:
bits = ['000111101011001','100011', '1000001', '00101001110110', '0001101',
        '00011110101100100011110101100']

for i in bits:
    pg = postGolomb(i)
    if(pg):
        print(f'({i}) cumple con los postulados de Golomb')
    else:
        print(f'({i}) no cumple con los postulados de Golomb')

(000111101011001) cumple con los postulados de Golomb
(100011) no cumple con los postulados de Golomb
(1000001) no cumple con los postulados de Golomb
(00101001110110) no cumple con los postulados de Golomb
(0001101) cumple con los postulados de Golomb
(00011110101100100011110101100) no cumple con los postulados de Golomb


## Linear Feedback Shift Register (LSFR)
**LSFR** is a shift register, in which the input is a bit that comes from applying linear transform function to a previous state.

The initial value is called seed, and the way of operating is deterministic, on the other hand, the secuence of generated values are determined by current state and previous state.

### How does it work?
The function reveives the coefficients, seed and the length of the sequence to generate **L**.

Then we get the last **k bits** of the seed, where **k** is the length of the coefficients.

Example: 

If the coefficients are $c = x^{10} + x^9 + x + 1$ and the length of the coefficients is $k = 10$, then we will take the last 10 bits of the seed, since the coefficients in bits are **1100000001**.

Next, we multiply the last **k bits** of the binary seed by the binary coefficients and add all the bits to said result, obtaining a new bit that is added to the end of the seed sequence.

Example:
$$seed = 1010; \thinspace \thinspace coefficients = X^4 + X + 1 = 1001;$$
$$new\_bit = \sum{1010 * 1001} = \sum{1000} = 1$$

This operation is repeated until the length of the output sequence is equal to **L**, returning a cyclic sequence.

In [4]:
# L es la longitud de la secuencia 
def LFSR(coeficientes, semilla, L=0):
    
    #secuencia final a devolver, que contendra semilla + restoBitsGenerados
    secuencia_f = semilla[:]
    
    #contador total que tiene actualmente la secuencia de salida
    contador = len(secuencia_f)
    
    #bucle infinito
    while(True):
        
        '''
        genero el resto de bits con los coeficientes y los ultimos x valores
        de la semilla de longitud igual a la de los coeficientes.
        Ejemplo:
        # c -->  1001
        # s --> 01101 --> 1*1 + 0*1 + 0*0 + 1*1 = 0
        '''
        '''
        cojo los ultimos k bits de la semilla, siendo k la longitud de  los 
        coeficientes.
        '''
        ultimos = np.array(list(map(int,secuencia_f[-len(coeficientes):])))
        
        #calculo el nuevo bit
        bit = np.sum(ultimos * np.array(list(map(int,coeficientes))))%2
        
        #y lo añado a la secuencia
        secuencia_f += str(bit)
        
        #incremento el contador de que se ha añadido un nuevo bit
        contador += 1
        
        '''
        si L es positivo y mayor que 0, compruebo si la secuencia de salida 
        tiene longitud L, si lo tiene devuelvo la secuencia final
        '''
        if(L>0 and contador >= L):
            return secuencia_f
        elif(L<=0):
            #recorro toda la secuencia menos el ultimo bit que es el que hemos metido
            for i in np.arange(len(secuencia_f[len(coeficientes)-1:-2])):
                
                #y comparo si se ha repetido algun valor
                if(secuencia_f[i:i+len(coeficientes)] == secuencia_f[-4:]):
            
                    '''
                    En caso de que L es negativo o 0, compruebo si ha llegado al final de 
                    la secuencia ciclica, si lo es devuelvo la secuencia final.
                    '''
                    return secuencia_f

Some examples:

Case of a reducible polynomial and therefore not primitive.

$$C = x^{4} + x^{3} + x^{2} + 1$$

$$L = 7; S = 7$$

In such a way that as can be seen in the result below, it generates a cyclic sequence, but not a complete one:

- 7, 14, 13, 10, 4, 9, 3

But, what if the seed is 1?

$$L = 7; S = 1$$

We obtain another different cyclic sequence, with other values different from the previous sequence.

- 1, 2, 5, 11, 6, 12, 8

Do all the values obtained from the 2 sequences, we are missing 0 and 15, in this case the values are repeated:

$$L = 1; S = 0 \thinspace o \thinspace 15$$
- 0, 0, ...
- 15, 15, ...

This tells us that it depends on the seed, we get one sequence or another. On the other hand, as the polynomial is reducible, it can be reduced to $C = (x+1)(x^3 + x + 1)$, indicating that it has 2 degrees, 1 and 3, with which we obtain cyclic sequences of length $2^1-1 = 1$ and $2^3-1=7$.


In [5]:
#Ejemplo polinomio reducible y por tanto no primitivo
coeficientes = '1110' #x^4 + x^3 + x^2 + 1
semilla = ['0111', '0001', '1111', '0000']
L = [4+7, 7+4, 1+4, 1+4]

print('Ejemplo polinomio reducible y por tanto no primitivo:')
for i in np.arange(len(semilla)):
    secuencia = LFSR(coeficientes, semilla[i], L[i])

    print(f'Partiendo de los coeficientes {coeficientes}, semilla {semilla[i]}, y la longitud {L[i]}')
    #print(secuencia)
    print('Secuencia final: ',end='')
    for i in np.arange(0,len(secuencia[3:])):
        print(f'{secuencia[i:i+4]} ',end='')
    print('')

Ejemplo polinomio reducible y por tanto no primitivo:
Partiendo de los coeficientes 1110, semilla 0111, y la longitud 11
Secuencia final: 0111 1110 1101 1010 0100 1001 0011 0111 
Partiendo de los coeficientes 1110, semilla 0001, y la longitud 11
Secuencia final: 0001 0010 0101 1011 0110 1100 1000 0001 
Partiendo de los coeficientes 1110, semilla 1111, y la longitud 5
Secuencia final: 1111 1111 
Partiendo de los coeficientes 1110, semilla 0000, y la longitud 5
Secuencia final: 0000 0000 


Case of a irreducible polynomial and not primitive.
$$C = x^4 + x^3 + x^2 + x + 1$$
$$L = 5; S = 7$$

In such a way that as can be seen in the result  below, it generates a cyclic sequence, but not a complete one:

- 7, 15, 14, 13, 11

But, what if the seed is 1?

$$L = 5; S = 1$$

We obtain another different cyclic sequence, with other values different from the previous sequence, but with the same length.

- 1, 3, 6, 12, 8

And in the case of 2?

We obtain another cyclic sequence, with other values different from the previous sequences, but with the same length.

- 2, 5, 10, 4, 9

And 0?, in this case  the value 0 is repeated.

In this example, the polinomial is of degree 4, with which $2^4-1=15$, but in this case we obtain 3 sequences of length 5, with which $3 * 5 = 15$, it is curious, because the same thing happens with a reducible polynomial, but with the difference that all have the same length, not counting 0.

In [6]:
#Ejemplo polinomio irreducible y no primitivo
coeficientes = '1111' #x^4 + x^3 + x^2 + x + 1
semilla = ['0111' ,'0001', '0010', '0000']
L = [5+4, 5+4, 5+4, 1+4]

print('Ejemplo polinomio irreducible y no primitivo:')
for i in np.arange(len(semilla)):
    secuencia = LFSR(coeficientes, semilla[i], L[i])

    print(f'Partiendo de los coeficientes {coeficientes}, semilla {semilla[i]}, y la longitud {L[i]}')
    print('Secuencia final: ',end='')
    for i in np.arange(0,len(secuencia[3:])):
        print(f'{secuencia[i:i+4]} ',end='')
    print('')

Ejemplo polinomio irreducible y no primitivo:
Partiendo de los coeficientes 1111, semilla 0111, y la longitud 9
Secuencia final: 0111 1111 1110 1101 1011 0111 
Partiendo de los coeficientes 1111, semilla 0001, y la longitud 9
Secuencia final: 0001 0011 0110 1100 1000 0001 
Partiendo de los coeficientes 1111, semilla 0010, y la longitud 9
Secuencia final: 0010 0101 1010 0100 1001 0010 
Partiendo de los coeficientes 1111, semilla 0000, y la longitud 5
Secuencia final: 0000 0000 


Case of the a irreducible polynomial and primitive.

$$C = x^4+x+1$$
$$L = 15; S = 7$$

This polynomial regardless of the seed it gives us, without counting the 0, will return the same complete cyclic sequence, but starting from the seed it gives us:

- 7, 15, 14, 13, 10, 5, 11, 6, 12, 9, 2, 4, 8, 1, 3

With which, being of degree 4, the sequence will have a length of $2^4-1=15$, which coincides with the complete sequence.

If the seed is 0, in this case the value 0 is repeated.

In [7]:
#Ejemplo polinomio irreducible y primitivo
coeficientes = '1001' #x^4 + x + 1
semilla = ['0111', '0000']
L = [15+4, 1+4]

print('Ejemplo polinomio irreducible y primitivo:')
for i in np.arange(len(semilla)):
    secuencia = LFSR(coeficientes, semilla[i], L[i])

    print(f'Partiendo de los coeficientes {coeficientes}, semilla {semilla[i]}, y la longitud {L[i]}')
    print('Secuencia final: ',end='')
    for i in np.arange(0,len(secuencia[3:])):
        print(f'{secuencia[i:i+4]} ',end='')
    print('')

Ejemplo polinomio irreducible y primitivo:
Partiendo de los coeficientes 1001, semilla 0111, y la longitud 19
Secuencia final: 0111 1111 1110 1101 1010 0101 1011 0110 1100 1001 0010 0100 1000 0001 0011 0111 
Partiendo de los coeficientes 1001, semilla 0000, y la longitud 5
Secuencia final: 0000 0000 


## Nonlineal Feedback Shift Register (NLFSR)

This algorithm is a shift register whose input bit is a non-linear function of its previous state.

The algorithm needs a polinomial function, a seed and a positive integer.

We will represent these values as follows:

$$f = 1 + x + y +xy = (x^0y^0)+(x^1y^0)+(x^0y^1)+(x^1y^1)$$

Then the polynimial function is represented:
**f = [[00][10][01][11]]**

Give a seed **s = [11]** and a positive integer **k = 4**.

NLFSR calculates the new bits as follows:

$new\_bit = x^1 + y^1 + x^1y^1 = (0^10^1)+(1^10^1)+(0^11^1)+(1^11^1) = 1$

And it is added at the end of the seed.

In [8]:
def NLFSR(f, s, k):
    #f --> [[1,1], [1,0]] --> xy + x
    #s --> es la semilla [1,1]
    #la semilla tiene que terner la misma longitud que la función
    
    #copiamos la semilla en la secuencia final a devolver
    secuencia_final = s[:]
    
    #mientras la longitud de la secuencia final sea < k
    while(len(secuencia_final) < k):
        
        #reseteamos el nuevo bit a 0
        nuevo_bit = 0
        
        '''
        Cogemos lo n ultimos bits, siendo n la longitud de la semilla 
        que recibe que debe coincidir con la longitud de cada componente
        del polinomio.
        '''
        n_ultimos = secuencia_final[-len(s):]
        
        #recorremos cada componente del polinomio
        for i in f:
            
            '''
            Obtenemos el nuevo bit, ejemplo: bit = 0^1 * 1^1 mod 2 = 0,
            donde 01 es una componente de la función polinomica y 11 son
            los ultimos 2 bits de la secuencia.
            '''
            nuevo_bit += np.prod(np.power(n_ultimos,i))
            nuevo_bit %= 2
            
        #una vez calculado el nuevo bit se añade a la secuencia
        secuencia_final.append(nuevo_bit)
        
    #cuando tengamos la secuencia completa la devolvemos    
    return secuencia_final

Some examples:

In [9]:
f = [[0,1],[1,0]]
s = [1,1]
k=2+3
secuencia = NLFSR(f,s,k)
 
print(f'Dado el polinomio: {f}, la semilla: {s}, y k = {k}')
print(f'Obtenemos la secuencia: ')
for i in np.arange(0,len(secuencia[1:])):
        print(f'{secuencia[i:i+2]} ',end='')
print('')

 
f = [[1,0,0],[0,0,1]]
s = [1,0,1]
k=3+7
secuencia = NLFSR(f,s,k)

print(f'Dado el polinomio: {f}, la semilla: {s}, y k = {k}')
print(f'Obtenemos la secuencia: ')

for i in np.arange(0,len(secuencia[2:])):
        print(f'{secuencia[i:i+3]} ',end='')
print('')

Dado el polinomio: [[0, 1], [1, 0]], la semilla: [1, 1], y k = 5
Obtenemos la secuencia: 
[1, 1] [1, 0] [0, 1] [1, 1] 
Dado el polinomio: [[1, 0, 0], [0, 0, 1]], la semilla: [1, 0, 1], y k = 10
Obtenemos la secuencia: 
[1, 0, 1] [0, 1, 0] [1, 0, 0] [0, 0, 1] [0, 1, 1] [1, 1, 1] [1, 1, 0] [1, 0, 1] 


In [10]:
print('Ejemplo del ejercicio:')
f = [[0,0,0,0],[0,0,1,0],[1,1,1,0],[0,0,0,1]]
s = [1,0,1,1]
k=4+9
secuencia = NLFSR(f,s,k)
 
print(f'Dado el polinomio: {f}, la semilla: {s}, y k = {k}')
print(f'Obtenemos la secuencia: ')
for i in np.arange(0, len(secuencia[3:])):
    print(f'{secuencia[i:i+4]} ',end='')
print('')

Ejemplo del ejercicio:
Dado el polinomio: [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 1, 0], [0, 0, 0, 1]], la semilla: [1, 0, 1, 1], y k = 13
Obtenemos la secuencia: 
[1, 0, 1, 1] [0, 1, 1, 1] [1, 1, 1, 1] [1, 1, 1, 0] [1, 1, 0, 1] [1, 0, 1, 0] [0, 1, 0, 0] [1, 0, 0, 1] [0, 0, 1, 0] [0, 1, 0, 0] 


In this case it goes from seed 11 to 7, 7 to 15, 14 to 13, 13 to 10 and from 10 to 4 and remains in a cycle 4, 9, 2, being the sequence 11, 7, 15, 14, 13, 10, 4, 9, 2 and repeats the sequence 4, 9, 2, this is one of the reasons why NFLSR is not used.

![Si no visualizas la imagen, es porque la carpeta imagenes no se encuentra en el mismo directorio que este cuaderno.](imgs/nflsr.jpg "Graphic design of the Geffe generator (image taken from the book 'cryptography notes' page 80)")

## Geffe Generator

This generator is built from 3 LFSR, in which each one receives a polinomial, a seed and a length, these LFSRs return a sequence each.

![Si no visualizas la imagen, es porque la carpeta imagenes no se encuentra en el mismo directorio que este cuaderno.](imgs/generador_geffe.png "Graphic design of the Geffe generator (image taken from the book 'cryptography notes' page 80)") 

With these sequences obtained from the 3 LFSR, we apply the following equation:

$$F(x_1, x_2, x_3) = x_1*x_2 + (1 + x_2)x_3 = x_1x_2 + x_2x_3 + x_3 = x_1x_2 \oplus x_2x_3 \oplus x_3$$

Having a linear complexity of $L_1L_2 + L_2L_3 + L_3$, where $L_1$, $L_2$ and $L_3$ are the lengths of the sequences. And a period of $lcm(2^{L_1}-1, 2^{L_2}-1,2^{L_3}-1)$

In [11]:
def GenGeffe(s1, s2, s3):

    #obtenemos las 3 secuencias de los 3 LFSR
    secuencia1 = np.array(list(map(int, list(LFSR(s1[0], s1[1],s1[2])))))
    secuencia2 = np.array(list(map(int, list(LFSR(s2[0], s2[1], s2[2])))))
    secuencia3 = np.array(list(map(int, list(LFSR(s3[0], s3[1], s3[2])))))
    
    '''
    resultado = (S1 * S2) + (S2 * S3) + S3 = (S1 * S2) XOR (S2 * S3) XOR S3
    Ambos son equivalentes, pero en mi caso utilizo el operador XOR, ya que es 
    más eficiente.
    '''
    secuencia_final = ((secuencia1 * secuencia2) ^ (secuencia2 * secuencia3) ^ secuencia3) % 2
    #secuencia_final = ((secuencia1 * secuencia2) + (secuencia2 * secuencia3) + secuencia3) % 2
    
    return secuencia_final

Some examples:

In [12]:
s1 = ('1001', '0111',5+4)
s2 = ('1001', '0001',5+4)
s3 = ('1001', '1001',5+4)

r = GenGeffe(s1, s2, s3)

print(f'''Dado 3 secuencias:
      - polinomio: {s1[0]}, semilla: {s1[1]}, L = {s1[2]}
      - polinomio: {s2[0]}, semilla: {s2[1]}, L = {s2[2]}
      - polinomio: {s3[0]}, semilla: {s3[1]}, L = {s3[2]}

Nos da como resultado la siguiente secuencia:
{r}
      ''')

s1 = ('1001', '1011', 9+4)
s2 = ('10010', '01001',8+5)
s3 = ('1000001', '1000011',6+7)

r = GenGeffe(s1, s2, s3)

print(f'''Dado 3 secuencias:
      - polinomio: {s1[0]}, semilla: {s1[1]}, L = {s1[2]}
      - polinomio: {s2[0]}, semilla: {s2[1]}, L = {s2[2]}
      - polinomio: {s3[0]}, semilla: {s3[1]}, L = {s3[2]}

Nos da como resultado la siguiente secuencia:
{r}
      ''')
      

s1 = ('110001011101110101111010000100000101110110111110100001100010010101001', '100000000001000001100010110000100001110000101000111100001100000110000', 69+50)
s2 = ('110001011101110101111010000100000101110110111110100001100010010101001', '010011110001100000000000001000001110000001100100000000100100111000111', 69+50)
s3 = ('110001011101110101111010000100000101110110111110100001100010010101001', '111100011100000001111110000000101100000100000110100000010000110001001', 69+50)

r = GenGeffe(s1, s2, s3)

print(f'''Dado 3 secuencias:
      - S1:
          - polinomio: {s1[0]}
          - semilla: {s1[1]}
          - L = {s1[2]}
      - S2:
          - polinomio: {s2[0]}
          - semilla: {s2[1]}
          - L = {s2[2]}
      - S3:
          - polinomio: {s3[0]}
          - semilla: {s3[1]}
          - L = {s3[2]}

Nos da como resultado la siguiente secuencia:
{r}
      ''')

Dado 3 secuencias:
      - polinomio: 1001, semilla: 0111, L = 9
      - polinomio: 1001, semilla: 0001, L = 9
      - polinomio: 1001, semilla: 1001, L = 9

Nos da como resultado la siguiente secuencia:
[1 0 0 1 1 0 1 1 1]
      
Dado 3 secuencias:
      - polinomio: 1001, semilla: 1011, L = 13
      - polinomio: 10010, semilla: 01001, L = 13
      - polinomio: 1000001, semilla: 1000011, L = 13

Nos da como resultado la siguiente secuencia:
[1 0 0 0 0 1 1 0 0 0 0 1 1]
      
Dado 3 secuencias:
      - S1:
          - polinomio: 110001011101110101111010000100000101110110111110100001100010010101001
          - semilla: 100000000001000001100010110000100001110000101000111100001100000110000
          - L = 119
      - S2:
          - polinomio: 110001011101110101111010000100000101110110111110100001100010010101001
          - semilla: 010011110001100000000000001000001110000001100100000000100100111000111
          - L = 119
      - S3:
          - polinomio: 110001011101110101111010000100000

### Stream cipher from Geffe generator

Here I am going to build a stream cipher, in which it receives the message as input **m** and returns the encrypted message **c**.

Internally, what it does is generate a key with a length equal to that of the message, using:
- Polynomial: $x^4+x+1$
- Seeds: 0111, 1000, 0001
- And L = length of the message

With this data we generate the key using the Geffe generator.

And finally we apply the XOR operation to the message with the key.

$$c = m \oplus k$$

To decrypt the encrypted message, just apply the same operation to the encrypted message.

$$m = c \oplus k$$

Since $m = m \oplus k \oplus k$, and the same $c = c \oplus k \oplus k$, with which $k \oplus k = 0$

In [13]:
def cifrado(m):
    
    #compruebo que la longitud del mensaje sea mayor a la de la semilla
    if(len(m)>4):
        
        #convierto m que es un string en un array de enteros
        m2 = np.array(list(map(int, list(m))))
        
        #estos son los polinomios, semillas y longitudes que les voy a pasar al generador
        s1 = ('1001','0111',len(m))
        s2 = ('1001','1000', len(m))
        s3 = ('1001','0001',len(m))
    
        #genero una clave k
        k = GenGeffe(s1, s2, s3)
        
        #convierto k que es un string en una lista de enteros
        k = np.array(list(map(int, list(k))))
        
        #aplico la operación m XOR k
        c = m2 ^ k
        
        #devueldo dicho resultado
        return ''.join(map(str,c))
    
    #si el mensaje tiene longitud < 4 devolvera -1
    return -1


def descifrado(c):
    
    #compruebo que la longitud del mensaje sea mayor a la de la semilla
    if(len(c)>4):
        
        #convierto c que es un string en un array de enteros
        c2 = np.array(list(map(int, list(c))))
        
        #estos son los polinomios, semillas y longitudes que les voy a pasar al generador
        s1 = ('1001','0111',len(c))
        s2 = ('1001','1000', len(c))
        s3 = ('1001','0001',len(c))
    
        #genero una clave k
        k = GenGeffe(s1, s2, s3)
        
        #convierto k que es un string en una lista de enteros
        k = np.array(list(map(int, list(k))))
        
        #aplico la operación c XOR k
        m = c2 ^ k
        
        #devueldo dicho resultado
        return ''.join(map(str,m))
    
    #si el mensaje cifrado tiene longitud < 4 devolvera -1
    return -1

Example:

In [14]:
m = '1110100101'

c = cifrado(m)

# =============================================================================
# para mostrar la clave por pantalla
s1 = ('1001','0111',len(m))
s2 = ('1001','1000', len(m))
s3 = ('1001','0001',len(m))
#genero una clave k
k = GenGeffe(s1, s2, s3)
# =============================================================================

print(f'Dado el mensaje: {m}, lo ciframos con la clave: {k}')
print(f'Cuya clave es generada utilizando el polinomio x^4 + x + 1, con las semillas {s1[1]}, {s2[1]}, {s3[1]}')
print(f'Obteniendo nuestro mensaje cifrado: {c}')

m = descifrado(c)

print(f'Y al descifrar el mensaje codificado obtenemos el mensaje original: {m}\n')
print(f'Ya que {k} XOR {k} = {k^k}')

Dado el mensaje: 1110100101, lo ciframos con la clave: [0 0 0 1 1 0 1 0 1 1]
Cuya clave es generada utilizando el polinomio x^4 + x + 1, con las semillas 0111, 1000, 0001
Obteniendo nuestro mensaje cifrado: 1111001110
Y al descifrar el mensaje codificado obtenemos el mensaje original: 1110100101

Ya que [0 0 0 1 1 0 1 0 1 1] XOR [0 0 0 1 1 0 1 0 1 1] = [0 0 0 0 0 0 0 0 0 0]


## Berlekamp Massey Algorithm

This is an algorithm that will find the shortest **linear-feedback shift register** (LFSR) for a given binary output sequence. The algorithm will also find the **minimal polinomial** of a linearly recurrent sequence in an arbitrary field.

## Sources:
- https://es.wikipedia.org/wiki/Postulados_de_Golomb
- https://es.wikipedia.org/wiki/LFSR
- https://upcommons.upc.edu/bitstream/handle/2117/193061/Gonzalo-Soriano.pdf?sequence=1&isAllowed=y (Pag 4)
- https://en.wikipedia.org/wiki/Nonlinear-feedback_shift_register
- https://aip.scitation.org/doi/pdf/10.1063/1.5132456 (Pag 3)
- https://tesis.ipn.mx/bitstream/handle/123456789/1357/FRANCISCOPERALTA.PDF?sequence=1&isAllowed=y
- https://www.scielo.cl/scielo.php?script=sci_arttext&pid=S0718-07642006000300023
- https://en.wikipedia.org/wiki/Berlekamp%E2%80%93Massey_algorithm
